In [1]:
import json
import time

from bs4 import BeautifulSoup
import pandas as pd
import requests

In [2]:
def find_gaps(data):
    """Calculate i) the span of min/max vintage years and ii) the number of vintage years credits have been issued"""
    delta_vintage = data['vintage'].max() - data['vintage'].min() + 1
    n_vintage = data['vintage'].nunique()
    return delta_vintage - n_vintage 

In [3]:
credits = pd.read_parquet('https://carbonplan-offsets-db.s3.us-west-2.amazonaws.com/final/2024-06-12/credits-augmented.parquet')
issuances = credits[(credits['transaction_type'] == 'issuance') & (credits['project_id'].str.startswith('CAR'))] # only interested in CAR projects here

projects = pd.read_parquet('https://carbonplan-offsets-db.s3.us-west-2.amazonaws.com/final/2024-06-12/projects-augmented.parquet')

In [4]:
gappy_subset = issuances.groupby('project_id').apply(find_gaps)
#gappy_proj_lst = gappy_subset[gappy_subset].index.values.tolist()

car_w_gaps = gappy_subset[gappy_subset >= 3].index.tolist()

Downloaded `car-v6-landfills` by going to https://thereserve2.apx.com/mymodule/mypage.asp.
Filter by `project type` == "landfill" and `protocol version` == 6.0. 


In [5]:
v6_landfills = pd.read_csv('data/car-v6-landfills.csv') 

In [6]:
gappy_v6_landfills = list(set(car_w_gaps) & set(v6_landfills['Project ID'].tolist()))
display(sorted(gappy_v6_landfills))

['CAR466', 'CAR473', 'CAR512', 'CAR514', 'CAR515', 'CAR516']

In [11]:
non_additional = v6_landfills[v6_landfills['Project ID'].isin(gappy_v6_landfills)]['Total Offset Credits Issued'].sum() 

In [12]:
total = v6_landfills['Total Offset Credits Issued'].sum()

In [13]:
display(f"Non-additional credits: {non_additional / 1_000_000:.2f} million")

'Non-additional credits: 0.41 million'

In [14]:
display(f"Total issuance v6 landifll: {total / 1_000_000:.2f} million")

'Total issuance v6 landifll: 0.84 million'

In [15]:
display(f"Percent over-crediting: {non_additional / total*100:.1f} percent")

'Percent over-crediting: 49.3 percent'

In [16]:
n_projects = v6_landfills['Project ID'].nunique()
display(f"Number of credited v6 projects: {n_projects}")

'Number of credited v6 projects: 14'

Get EPA FLIGHT Data. 
The first step is to find the entity id for each project.
This involved searching for projects by name in the tool.
For all projects, we confirmed the address on project paperwork matched the address of the landfill in the EPA database.

In [17]:
project_epa_map = {
    "CAR515": 1003621,
    "CAR516": 1007808,
    "CAR512": 1005900,
    "CAR466": 1002863,
    "CAR514": 1005730,
    "CAR473": 1007282,
    
}

In [18]:
def get_vintage_years(data):
    """Get years project has received credits"""
    return {"credit_data": data[(data['transaction_type'] == 'issuance') & (data['vintage'] >=2010)].vintage.unique().tolist()}

def get_flight_data(landfill_id, year):
    """Download and extract data by entity id and year"""
    url = f"https://ghgdata.epa.gov/ghgp/service/xml/{year}?id={landfill_id}&et=undefined"
    r = requests.get(url)
    soup = BeautifulSoup(r.content)
    record = {}
    record['n_wells'] = int(soup.find('numberofwells').text)
    record['gccs_capacity'] = int(soup.find('systemcapacity').find('measurevalue').text)
    record['active_gccs'] = True if soup.find('islandfillgascollectionsystemindicator').text == 'Y' else False
    record['year'] = int(soup.find('reportingyear').text) # just to make sure iterating correctly
    time.sleep(1) # be kind to the endpoint
    return record

In [19]:
records = issuances[issuances.project_id.isin(project_epa_map.keys())].groupby('project_id').apply(get_vintage_years).to_dict()

In [20]:
flight_data = {}
for project_id in records.keys():
    years = range(2010, 2023)
    flight_data[project_id] = {year: get_flight_data(project_epa_map[project_id], year) for year in years}

In [21]:
for project_id, facility_data in flight_data.items():
    active_gccs = [k for k,v in facility_data.items() if v['active_gccs']]
    records[project_id]['epa_data'] = active_gccs

In [22]:
display(records)

{'CAR466': {'credit_data': [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2022],
  'epa_data': [2010,
   2011,
   2012,
   2013,
   2014,
   2015,
   2016,
   2017,
   2018,
   2019,
   2020,
   2021,
   2022]},
 'CAR473': {'credit_data': [2010, 2011, 2018, 2019, 2020, 2021, 2022],
  'epa_data': [2010,
   2011,
   2012,
   2013,
   2014,
   2015,
   2016,
   2017,
   2018,
   2019,
   2020,
   2021,
   2022]},
 'CAR512': {'credit_data': [2010,
   2011,
   2012,
   2013,
   2014,
   2015,
   2016,
   2017,
   2021,
   2022,
   2023],
  'epa_data': [2010,
   2011,
   2012,
   2013,
   2014,
   2015,
   2016,
   2017,
   2018,
   2019,
   2020,
   2021,
   2022]},
 'CAR514': {'credit_data': [2010, 2011, 2012, 2022],
  'epa_data': [2010,
   2011,
   2012,
   2013,
   2014,
   2015,
   2016,
   2017,
   2018,
   2019,
   2020,
   2021,
   2022]},
 'CAR515': {'credit_data': [2010, 2011, 2022],
  'epa_data': [2010,
   2011,
   2012,
   2013,
   2014,
   2015,
   2016,
   2017,
   2018,
   2019,
 

In [ ]:
with open('/tmp/gappy-landfill-data.json', 'w') as f: # for frontend
    json.dump(records, f)